### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20220921_scenario_projections.ipynb")

2022/09/28 17:31:59 INFO|============================================================


### Fit an initial scenario through February 2022

In [3]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20220920_sep_cdphe_slides/20220920_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/20220920_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

2022/09/28 17:32:02 INFO|-------------------------|{} Retrieving vaccinations data
2022/09/28 17:32:05 INFO|-------------------------|{} Constructing vaccination projections
2022/09/28 17:32:07 INFO|-------------------------|{} Retrieving hospitalizations data
2022/09/28 17:32:10 INFO|----------------------|{} Prepping Model
2022/09/28 17:32:10 INFO|-----------------------|{} Prepping Model
2022/09/28 17:33:20 INFO|----------------------|{} Model prepped for fitting in 70.17418170900001 seconds.
2022/09/28 17:33:20 INFO|----------------------|{} Will fit 25 times
2022/09/28 17:34:21 INFO|----------------------|{'fit_batch': '0'}: Transmission control fit 1/25 completed in 61.546623334 seconds: {0: {'co': 0.8587936417173216}, 14: {'co': 0.6896751305823303}, 28: {'co': 0.5521636576482694}, 42: {'co': 0.0012931829830483534}, 56: {'co': 0.6815575142330268}, 70: {'co': 0.8912296555117708}}
2022/09/28 17:34:21 INFO|-----------------------|{'fit_batch': '0'}: Running forward sim
2022/09/28 17

2022/09/28 17:39:15 INFO|----------------------|{'fit_batch': '18'}: Transmission control fit 19/25 completed in 11.728665250000006 seconds: {504: {'co': 0.7861769356332884}, 518: {'co': 0.7799123275231242}, 532: {'co': 0.7570420570288318}, 546: {'co': 0.7091659965869969}, 560: {'co': 0.7430642132060428}, 574: {'co': 0.7376034186082499}}
2022/09/28 17:39:16 INFO|-----------------------|{'fit_batch': '18'}: Running forward sim
2022/09/28 17:39:35 INFO|----------------------|{'fit_batch': '19'}: Transmission control fit 20/25 completed in 17.466870583000002 seconds: {532: {'co': 0.756216654923652}, 546: {'co': 0.7112514576383564}, 560: {'co': 0.7393367941617954}, 574: {'co': 0.7492594382632967}, 588: {'co': 0.786248013613611}, 602: {'co': 0.7652557875479273}}
2022/09/28 17:39:35 INFO|-----------------------|{'fit_batch': '19'}: Running forward sim
2022/09/28 17:39:48 INFO|----------------------|{'fit_batch': '20'}: Transmission control fit 21/25 completed in 11.132708541999989 seconds: {

### Create and run scenarios from Feb 2022 to present

In [9]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20220920_sep_cdphe_slides/20220920_scenario_params.json"))

model_args = {
    'base_spec_id': 4055, #model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 1]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
                   "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-20": vx_seed, "2022-10-20": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                                "attrs": {'age': '0-19', 'vacc': 'shot1'},
                                "mults": {"2020-01-01": 1, "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                          "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                          "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                          "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                          "2022-07-30": 0.955 + 0.045*vacc_eff_lt5,
                                          "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                          "2022-08-20": 0.945 + 0.055*vacc_eff_lt5,
                                          "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                          "2022-09-10": 0.935 + 0.065*vacc_eff_lt5,
                                          "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                                "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                        'hosp_reporting_frac': hrf,
                                        'tags': {'vx_seed': vx_seed,
                                                 'vir_mult': vir_mult,
                                                 'booster_mult': 0}})
            

In [10]:
# check how many scenarios there are
len(scenario_model_args)

4

In [11]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/09/28 19:51:04 INFO|============================================================
2022/09/28 19:51:04 INFO|============================================================
2022/09/28 19:51:04 INFO|============================================================
2022/09/28 19:51:04 INFO|============================================================
2022/09/28 19:51:08 INFO|-----------|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/09/28 19:51:08 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/09/28 19:51:08 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/09/28 19:51:08 INFO|-----------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/09/28 19:51:12 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0} Constructing vaccination projections
2022/09/28 19:51:12 INFO|-----------|{'vx_seed': 1, 'vi

2022/09/28 19:53:22 INFO|--------|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 13.326450333999986 seconds: {823: {'co': 0.848186685247527}, 837: {'co': 0.8461261938797114}, 851: {'co': 0.832656830072356}, 865: {'co': 0.8616560313693256}, 879: {'co': 0.8533330440424136}, 893: {'co': 0.8575286762306822}}
2022/09/28 19:53:22 INFO|---------|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Running forward sim
2022/09/28 19:53:23 INFO|--------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 13.24757516599999 seconds: {823: {'co': 0.848186685247527}, 837: {'co': 0.8461261938797114}, 851: {'co': 0.832656830072356}, 865: {'co': 0.8616560313693256}, 879: {'co': 0.8533330440424136}, 893: {'co': 0.8575286762306822}}
2022/09/28 19:53:23 INFO|---------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '2'}: Running forward sim
202

2022/09/28 19:53:58 INFO|--------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4'}: Transmission control fit 5/5 completed in 16.129970541999995 seconds: {879: {'co': 0.8541575585625053}, 893: {'co': 0.8550527782269102}, 907: {'co': 0.8610803151337563}, 921: {'co': 0.8631809374000383}, 935: {'co': 0.8484079917699039}, 949: {'co': 0.8506459181077451}}
2022/09/28 19:53:58 INFO|---------|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4'}: Running forward sim
2022/09/28 19:53:59 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8587936407411457}, 14: {'co': 0.6896751339489205}, 28: {'co': 0.5463057169481709}, 42: {'co': 1.4470575638442843e-14}, 56: {'co': 0.6964655911772399}, 70: {'co': 0.8333061588295956}, 84: {'co': 0.8295950515012411}, 98: {'co': 0.8317180033595475}, 112: {'co': 0.8774455019880499}, 126: {'co': 0.8243715269502497}, 140: {'co': 0.7604769120446827}, 154:

### Run the report for each fit model

In [12]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir,
                           prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/09/28 19:57:03 INFO|============================================================
2022/09/28 19:57:03 INFO|--------|Solving model
2022/09/28 19:57:03 INFO|============================================================
2022/09/28 19:57:03 INFO|--------|Solving model
2022/09/28 19:57:04 INFO|============================================================
2022/09/28 19:57:04 INFO|--------|Solving model
2022/09/28 19:57:05 INFO|============================================================
2022/09/28 19:57:05 INFO|--------|Solving model
2022/09/28 19:57:07 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/28 19:57:08 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/09/28 19:57:08 WARNING|No artists with labels found to put in legend.  Note that artists 

In [13]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/09/28 20:01:38 INFO|Projecting
2022/09/28 20:01:38 INFO|
2022/09/28 20:01:58 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 20:01:59 INFO|
2022/09/28 20:02:19 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 20:02:21 INFO|
2022/09/28 20:02:40 INFO|{'vx_seed': 1, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 20:02:42 INFO|
2022/09/28 20:03:03 INFO|{'vx_seed': 1, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/09/28 20:03:04 INFO|Running reports
